# Openai Embedding

## Api reference

### [Embedding](https://platform.openai.com/docs/api-reference/embeddings)

Get a vector representation of a given input that can be easily consumed by machine learning models and algorithms.
Related guide: [Embeddings](https://platform.openai.com/docs/guides/embeddings)
   
### Embedding object

Represents an embedding vector returned by embedding endpoint.

1. index (integer): The index of the embedding in the list of embeddings.
2. object (string): The object type, which is always "embedding".
3. embedding (array): The embedding vector, which is a list of floats. The length of vector depends on the model as listed in the [embedding guide](https://platform.openai.com/docs/guides/embeddings).

### Create embeddings

POST https://api.openai.com/v1/embeddings.
Creates an embedding vector representing the input text.

#### Request body

1. model (string) (Required): ID of the model to use. You can use the [List models](https://platform.openai.com/docs/api-reference/models/list) API to see all of your available models, or see our [Model overview](https://platform.openai.com/docs/models/overview) for descriptions of them.
2. input (string or array) (Required): Input text to embed, encoded as a string or array of tokens. To embed multiple inputs in a single request, pass an array of strings or array of token arrays. Each input must not exceed the max input tokens for the model (8191 tokens for text-embedding-ada-002) and cannot be an empty string. [Example Python code](https://github.com/openai/openai-cookbook/blob/main/examples/How_to_count_tokens_with_tiktoken.ipynb) for counting tokens.
3. user (string) (Optional): A unique identifier representing your end-user, which can help OpenAI to monitor and detect abuse. [Learn more](https://platform.openai.com/docs/guides/safety-best-practices/end-user-ids).

#### Returns

A list of embedding objects.

## 环境准备

### 安装所需依赖组件

> %pip will install the package in the virtual environment where the current notebook kernel is running. 
> While !pip will install the package in the base environment. 
> If you are using Python virtual environment (as you should!), you should use %pip.

前置通过 shell 命令，安装所依赖的 python 包

In [3]:
%pip install tiktoken openai pandas matplotlib plotly scikit-learn numpy

Note: you may need to restart the kernel to use updated packages.


### 导入依赖模块

导入本项目所依赖的所有模块。

此处发现有模块缺失，可以填补在上述命令中进行安装。

In [11]:
import os
import json
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import ast
import subprocess

# 导入 tiktoken 库。Tiktoken 是 OpenAI 开发的一个库，用于从模型生成的文本中计算 token 数量。
import tiktoken

# 从 openai.embeddings_utils 包中导入 get_embedding 函数。
# 这个函数可以获取 GPT-3 模型生成的嵌入向量。
# 嵌入向量是模型内部用于表示输入数据的一种形式。
# cosine_similarity 函数计算两个嵌入向量之间的余弦相似度。
import openai
from openai.embeddings_utils import get_embedding, cosine_similarity

# 从 sklearn.manifold 模块中导入 TSNE 类。
# TSNE (t-Distributed Stochastic Neighbor Embedding) 是一种用于数据可视化的降维方法，尤其擅长处理高维数据的可视化。
# 它可以将高维度的数据映射到 2D 或 3D 的空间中，以便我们可以直观地观察和理解数据的结构。
from sklearn.manifold import TSNE

# 从 scikit-learn中导入 KMeans 类。KMeans 是一个实现 K-Means 聚类算法的类。
from sklearn.cluster import KMeans

### 加载配置信息

该项目中，密钥配置在了根目录的 config.json 文件中，可替换为自己的密钥信息。

如使用 git 进行管理，请手动忽略该文件相关变更，避免信息泄露。

In [12]:
# 通过 subprocess 执行 shell 命令，获取 git 仓库的根目录
command = ['git', 'rev-parse', '--show-toplevel']
process = subprocess.Popen(command, stdout=subprocess.PIPE)
output, error = process.communicate()
git_root = output.decode().strip()
config_path = os.path.join(git_root, "config.json")

config = {}
with open(config_path,"r") as f:
    config = json.load(f)
openai.api_key = config["sk"]

## 获取 Embedding 数据

此步骤为非必要步骤，可直接使用项目中已经生成过的文件信息。

### 配置 Embedding 模型信息

In [29]:
# 模型类型
# 建议使用官方推荐的第二代嵌入模型：text-embedding-ada-002
embedding_model = "text-embedding-ada-002"
# text-embedding-ada-002 模型对应的分词器（TOKENIZER）
embedding_encoding = "cl100k_base"
# text-embedding-ada-002 模型支持的输入最大 Token 数是 8191，向量维度 1536
# 在我们的 DEMO 中过滤 Token 超过 8000 的文本
max_tokens = 100

### 加载数据集

> Source:[美食评论数据集](https://www.kaggle.com/snap/amazon-fine-food-reviews)

数据集选择亚马逊美食评论数据集(amazon-fine-food-reviews)，该数据集包含截至 2012 年 10 月用户在亚马逊上留下的共计 568,454 条美食评论。

为了说明目的，我们将使用该数据集的一个子集（/data/fine_food_reviews_1k.csv），其中包括最近 1,000 条评论。这些评论都是用英语撰写的，并且倾向于积极或消极。每个评论都有一个产品ID、用户ID、评分、标题（摘要）和正文。

我们将把评论摘要（Summary）和正文（Text）合并成一个单一的组合文本（combined）。模型将对这个组合文本进行编码，并输出一个单一的向量嵌入。

In [30]:
input_datapath = os.path.join(git_root, "openai-api", "data", "fine_food_reviews_1k.csv")

df = pd.read_csv(input_datapath, index_col=0)
df = df[["Time", "ProductId", "UserId", "Score", "Summary", "Text"]]
df = df.dropna()

# 将 "Summary" 和 "Text" 字段组合成新的字段 "combined"
df["combined"] = (
    "Title: " + df.Summary.str.strip() + "; Content: " + df.Text.str.strip()
)

### 处理数据集

在实际项目中，数据集本身会比较庞大，且文本信息可能会超过最大 token 限制，出于资源消耗考虑，可能会对数据集本身做一些精简处理。

In [31]:
# 设置要筛选的评论数量为1000
top_n = 100
# 对DataFrame进行排序，基于"Time"列，然后选取最后的2000条评论。
# 这个假设是，我们认为最近的评论可能更相关，因此我们将对它们进行初始筛选。
df = df.sort_values("Time").tail(top_n * 2) 
# 从'embedding_encoding'获取编码
encoding = tiktoken.get_encoding(embedding_encoding)

# 计算每条评论的token数量。我们通过使用encoding.encode方法获取每条评论的token数，然后把结果存储在新的'n_tokens'列中。
df["n_tokens"] = df.combined.apply(lambda x: len(encoding.encode(x)))

# 如果评论的token数量超过最大允许的token数量，我们将忽略（删除）该评论。
# 我们使用.tail方法获取token数量在允许范围内的最后top_n（1000）条评论。
df = df[df.n_tokens <= max_tokens].tail(top_n)

# 打印出剩余评论的数量。
len(df)

100

### 调用 Embedding 模型

In [32]:
# 实际生成会耗时几分钟
# 提醒：非必须步骤，可直接复用项目中的嵌入文件 fine_food_reviews_with_embeddings_1k
df["embedding"] = df.combined.apply(lambda x: get_embedding(x, engine=embedding_model))

output_datapath = input_datapath = os.path.join(git_root, "openai-api", "data", "fine_food_reviews_1k_with_embeddings.csv")

df.to_csv(output_datapath)